In [1]:
import numpy as np
from javaconnectormulti import AssetMaintenanceModelMultiRunConnector
from jpype import *
import random
import numpy as np
# For dates in graphs
import datetime
from matplotlib.dates import DateFormatter
# Import for graphs
import matplotlib.pyplot as plt
# generate random integer values
from numpy.random import seed
from numpy.random import randint
#to generate individual and population Via DEAP
from deap import base
from deap import creator
from deap import tools
import time

In [2]:
##############No need to change variables below here ####################

#Using Deap don't change the pop size here only use 1. Population is defined below in main code.
popSize=1
#Set binary size of Events
eventBinSize=5
#Set binary size of Assets
assetBinSize=4
#Always create this one time enviromet to run multiple times of the simulation


######Function to run Simulation######
def runSim(assetPerEvent,eventPerYear, ammc):
    #NOTE: you can create only one virtual enviroment.
    #ammc = AssetMaintenanceModelMultiRunConnector()

    #How we select what sheet to use. Ensure Excel sheet Asset Maintance Tab is left blank and that required posting
    #policy and workforce compostion is selected. See last semester for comparison.
    ammc.load_config( "conf/ThreeYearPostingNoAssets.xlsx" )

    # make local variables
    # required in order to setup helper function to populate data in the Asset Maintenance Tab
    helper = ammc.origin_conf
    config = helper.getRootConfig()

    AssetsMaintenanceConfig = JClass( "edu.unsw.csc.rbm.gm.utility.config.AssetsMaintenanceConfig" )
    ms = JClass( "edu.unsw.csc.rbm.gm.config.Asset_Maintenance_InfoScheduleConfig" )

    # create a object for the schedule in The Asset Maintenance Tab.
    # note that the excel sheet used needs this tab to be left blank. Check Semester 1 data for validity.
    # also 
    asset_id = "TESTASSET"
    start_usage_str = "EXERCISE"
    end_usage_str = "EXERCISE"
    end_facility_str = "STORAGE1"
    maint_facility_str = "FACILITY1"
    start_facility_str = "STORAGE1"

    maintCfg = helper.createMaintenanceConfig(asset_id, start_usage_str, end_usage_str, end_facility_str, maint_facility_str, start_facility_str)

    # manipultae number and per year in Pyhton
    numEventsPerYear = eventPerYear # this is the per year
    numAssetsPerEvent = assetPerEvent # this is the number

    #Populate the first line in the Asset Maintenace Schedule Tab as per above variables
    helper.addAssetConfigs(maintCfg, "01/01/2019", numEventsPerYear[0], numAssetsPerEvent[0] )
    #print("b")
    # Run once , get back results
    main = ammc.run_once()

    #Setup to get data
    ac = main.rootFactor.section_asset_module.get_from_asset_classes("TESTASSET")

    #Get the required data. Note that this can be altered to be monthly or quartley. Better to leave at lowest possible
    #rresolution to get best data.
    dataset1 = ac.ds_RFI

    #Workout Results
    RFI=[]
    for i in range(dataset1.getData().size()):
        RFI.append(dataset1.getData().get(i).yval)

    # take mean from the 8th vlue until the last which is 522.
    meanRFI=np.mean(RFI[20:522])
    
    #Get the maximum and minimum values to work out the range
    minRFI=min(RFI[20:522])
    maxRFI=max(RFI[20:522])
    rangeRFI=maxRFI-minRFI
    
    #print("Mean Value: %s" %(meanRFI))
    #print("Range Value: %s" %(rangeRFI))
    
    #With these numbers return a fitness value
    fitnessValue = meanRFI-(rangeRFI/5)
    return fitnessValue

###########Function To Run Sim ends###################

##########Run Sim For entire Population Start###############

def runSimForPop(indv):
    #Check what the binary value is that is being worked on
    #print(indv) #Lets you see what value the sim is being run on
    results=[]
    #Get the assets and events from the binary number
    assetForSim=[]
    eventForSim=[]
    #for x in range(len(pop)):
    temp=binToInt(getAssetBinElement(indv))
    temp2=binToInt(getEventBinElement(indv))
        #This makes sure that it is added into the relevant array in the proper way to run the function
    assetForSim.append(temp[0])
    eventForSim.append(temp2[0])
    #print(assetForSim) #Lets you see what the binary number is for the assets per event
    #print(eventForSim) #Lets you see what the binary number is for the events per year
    
    #get the fitness function through the simulation
    temp=runSim(assetForSim,eventForSim,ammc)
    
    #print(temp) #Print the resultant fitness value
    
    #Return the firness value
    return temp,
    
##########Run Sim For entire Population End###############

##########Functions below Here#######################

# Function to create events per year dependant on pop size
# Had to change it from 1 to 32 as 0 would case issues in the code
def createEventValues():
    noOfEvents = randint(1, 32, popSize)
    return noOfEvents

# Function to create assets per event dependant on pop size
# Had to change it from 1 to 15 as 0 would case issues in the code
def createAssetValues():
    noOfAssets = randint(1, 15, popSize)
    return noOfAssets

# Convert integer to binary number.
# input (integer,binary no size)
def convertToBin(number,binSize):
    binNumbersEvents=[]
    for x in number:
    #convert to binary and remove 0b at start and make sure they are all the same length (zfill)
        temp=bin(x)[2:].zfill(binSize)
        binNumbersEvents.append(temp)
    return binNumbersEvents

# Add an asset binary number to event binary number.
# input in binary form (asset,event)
def addBinElemtsAE(asset,event):
    # making a for loop to append the two numbers together
    popAppend=[]
    for x in range(len(asset)):
        temp=asset[x]+event[x]
        popAppend.append(temp)
    return popAppend

# Returns the binary number associated with the asset
# input (large binary string)
def getAssetBinElement(element):
    assetsNoToUse=[]
    for x in range(len(element)):
        #get the first element in the array
        tempChange=element[x]
        #get the first digits associated with the assets
        tempAsset=tempChange[:assetBinSize]
        #Append the assets
        assetsNoToUse.append(tempAsset)
    return assetsNoToUse

# Returns the binary number associated with the event
# input (large binary string)
def getEventBinElement(element):
    eventNoToUse=[]
    for x in range(len(element)):
        #get the first element in the array
        tempChange=element[x]
        #get the last digits associated with the events
        tempEvent=tempChange[assetBinSize:len(tempChange)]
        #append the events
        eventNoToUse.append(tempEvent)
    return eventNoToUse

# converts binary numbers into integers
# input (binary number)
def binToInt(binNo):
    
    intNo=[]
    # for the length of the binary array of events
    for x in range(len(binNo)):
        # get the first binary element
        temp=binNo[x]
        # convert to integer
        temp=int(temp, 2)
        ##CATCH THE BAD BIT####################
        if temp==0:
            temp=1
        ##END CATCH THE BAD BIT#################
        # add it to the list
        intNo.append(temp)
    return intNo

##############Functions End##########################


In [3]:
###############Added Stuff for DEAP#####################

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax) 
def generateIndividual():
    
    assetIntValues=createAssetValues()
    eventIntValues=createEventValues()

    assetBinValues=convertToBin(assetIntValues,assetBinSize)
    eventBinValues=convertToBin(eventIntValues,eventBinSize)
    combinedBinValues=addBinElemtsAE(assetBinValues,eventBinValues)
    
    ###WANT TO RUN OUTSIDE OF TOOLBOX REMOVE COMMA AT THE END!!!!!#####
    #Above is an old hangover note
    return creator.Individual(combinedBinValues)

##############Code to create first random popultaion and then run simulation here########################

#Now create a population
toolbox = base.Toolbox()
toolbox.register("individual", generateIndividual)
toolbox.register("evaluate", runSimForPop)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("select", tools.selTournament, tournsize=3)

def hasan_crossover(parent1, parent2):
    
    #print("Parent 1 is %s" % parent1) #Prints what the first parent is
    #print("Parent 2 is %s" % parent2) #Prints what the second parent is
    
    a,b=tools.cxOnePoint(list(parent1[0]), list(parent2[0]))
    
    child1=''.join([str(elem) for elem in a])
    child2=''.join([str(elem) for elem in b])
    
    #print("Child 1 is %s" % child1) #Prints the first crossover child
    #print("Child 2 is %s" % child2) #Prints the second crossover child
    
    parent1[:]=[child1] #Populate the global values
    parent2[:]=[child2] #Populate the global values
    
    return parent1, parent2

#     return ([child1],[child2])

toolbox.register("mate", hasan_crossover)

def sim_mutant (parent1):
    
    string_revised=" ".join(parent1[0])
    a_list = string_revised.split()
    map_object = map(int, a_list)
    list_of_integers = list(map_object)
    
    #Increased the bit being flipped from 0.1 to 0.15
    a=tools.mutFlipBit(list_of_integers[0:len(list_of_integers)], indpb=0.15) #Change chance of a bit being flipped
    
    #print (a[0]) #Print the value that will have crossover work on it
    
    child1=''
    
    for elem in a[0]:
        
        child1=child1+str(elem)
        
    #print(child1) #Print the mutated child
    
    parent1[:]=[child1] #Global Change
    
    return parent1

toolbox.register("mutate", sim_mutant)

###########population size defined here#####################
#populationSize=4
#pop = toolbox.population(n=populationSize)


ammc = AssetMaintenanceModelMultiRunConnector()    

In [4]:
#Setup for keeping track of values

bestFit=[]
betsInd=[]
generation=[]
meanList=[]

start_time = time.time() #start time

In [5]:
def main():
           
    pop = toolbox.population(n=50) #Change the population here

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual #0.5, 0.2
    # Origanlly set as CXPB, MUTPB = 0.0, 1.2
    CXPB, MUTPB = 0.5, 0.2
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while g < 100: #Change the length of generation here
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                #print('Crossing') #Let you know when a crossover is happening

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                #print("mutating") #Let you know when something is being mutated
                toolbox.mutate(mutant)
                #print("mutated") #Let you know when a mutation has happened
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        #print(invalid_ind) #Prints out the list of individuals evaluated 
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        #print(offspring)
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        #print(pop)
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        #Add the fitness values to a list
        best_ind = tools.selBest(pop, 1)[0]
       
        generation.append(g)
        bestFit.append(best_ind.fitness.values)
        betsInd.append(best_ind)
        meanList.append(mean)
                
        #print(pop)
    
    print("-- End of (successful) evolution --")
    stop_time = time.time() - start_time
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    

In [6]:
if __name__ == "__main__":
    main()

Start of evolution
  Evaluated 50 individuals
-- Generation 1 --
  Evaluated 29 individuals
  Min 3.6191235059760953
  Max 14.36414342629482
  Avg 9.942988047808766
  Std 2.051653213445076
-- Generation 2 --
  Evaluated 32 individuals
  Min 1.144621513944223
  Max 14.13824701195219
  Avg 9.846286852589643
  Std 2.4856103374181835
-- Generation 3 --
  Evaluated 36 individuals
  Min 0.2999999999999998
  Max 14.13824701195219
  Avg 10.791928286852592
  Std 2.8781695893640236
-- Generation 4 --
  Evaluated 25 individuals
  Min 0.6019920318725099
  Max 13.802788844621514
  Avg 10.63221513944223
  Std 3.473812880313178
-- Generation 5 --
  Evaluated 32 individuals
  Min 1.4912350597609563
  Max 14.36414342629482
  Avg 10.781394422310763
  Std 3.788627257370883
-- Generation 6 --
  Evaluated 33 individuals
  Min 0.12470119521912348
  Max 14.36414342629482
  Avg 10.806087649402393
  Std 4.02564986562686
-- Generation 7 --
  Evaluated 30 individuals
  Min 0.6115537848605577
  Max 14.36414342629

  Evaluated 33 individuals
  Min 2.433466135458167
  Max 14.36414342629482
  Avg 13.461290836653397
  Std 2.3158807369860193
-- Generation 59 --
  Evaluated 33 individuals
  Min 7.285657370517929
  Max 14.36414342629482
  Avg 13.766972111553796
  Std 1.7512135445424968
-- Generation 60 --
  Evaluated 29 individuals
  Min 2.385258964143426
  Max 14.36414342629482
  Avg 12.888326693227102
  Std 2.7728574573450766
-- Generation 61 --
  Evaluated 30 individuals
  Min 0.4709163346613545
  Max 14.36414342629482
  Avg 13.434183266932282
  Std 2.8192673632665306
-- Generation 62 --
  Evaluated 39 individuals
  Min 2.385258964143426
  Max 14.36414342629482
  Avg 13.09976095617531
  Std 2.6891653794308357
-- Generation 63 --
  Evaluated 36 individuals
  Min 1.144621513944223
  Max 14.36414342629482
  Avg 13.486996015936267
  Std 2.557113748310372
-- Generation 64 --
  Evaluated 29 individuals
  Min 2.385258964143426
  Max 14.36414342629482
  Avg 12.996326693227102
  Std 2.952659439180454
-- Gene

In [7]:
#Print the list of values for each generation
#print(generation)
stop_time = time.time() - start_time

print(bestFit)
print(betsInd)
print(meanList)

print(stop_time/60)

[(14.36414342629482,), (14.13824701195219,), (14.13824701195219,), (13.802788844621514,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.36414342629482,), (14.3641

In [8]:
# Make sure Results above have been copied to excel 
# Importing library
import csv
  
# opening the csv file in 'w+' mode
file = open('ListOfBestFitnessValuesThreeYearPostingAgain.csv', 'w+', newline ='')
  
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(bestFit)
    
import pandas as pd
  
# List initialization
  
df = pd.DataFrame()

df['Mean'] = meanList
  
# Converting to excel
df.to_excel('listOfMeanResultsThreeYearPostingAgain.xlsx', index = False)
    
print("Done")

Done


In [ ]:
##### LEFT OVER BELOW HERE #######

In [ ]:
# # opening the csv file in 'w+' mode
# file = open('ListOfPopMeanValues.csv', 'w+', newline ='')

# my_array = np.linspace(meanList)

# # writing the data into the file
# with file:    
#     write = csv.writer(file)
#     write.writerows(my_array)

# Importing pandas as pd
import pandas as pd
  
# List initialization
  
df = pd.DataFrame()

df['Mean'] = meanList
  
# Converting to excel
df.to_excel('listOfMeanResults.xlsx', index = False)
    
print("Done")